# the `BusyFlyerDevice`

It may not be practical to develop a mixin device for the ***BusyFlyer***.  When using a callback (much more efficient coding than with polling for completion), the steps to customize `kickoff()`, `complete()`, `collect()`, and `describe_collect()` comprise the major parts of these methods.  A general mixin class, providing hook methods to each of these parts is only extra code to maintain.  Better to provide a good template example.

In [1]:
import enum
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.from_config(databroker.temp_config())
RE.subscribe(db.insert)

0

In [2]:
BUSY_PV = 'prj:mybusy'
TIME_WAVE_PV = 'prj:t_array'
X_WAVE_PV = 'prj:x_array'
Y_WAVE_PV = 'prj:y_array'

In [3]:
class BusyStatus(str, enum.Enum):
    busy = "Busy"
    done = "Done"

class MyWaveform(ophyd.Device):
    """waveform records store fly scan data"""
    wave = ophyd.Component(ophyd.EpicsSignalRO, "")
    number_elements = ophyd.Component(ophyd.EpicsSignalRO, ".NELM")
    number_read = ophyd.Component(ophyd.EpicsSignalRO, ".NORD")

In [4]:
class BusyFlyerDevice(ophyd.Device):
    """
    support APS Fly Scans that are operated by a busy record
    """

    busy = ophyd.Component(ophyd.EpicsSignal, BUSY_PV, string=True)
    time = ophyd.Component(MyWaveform, TIME_WAVE_PV)
    axis = ophyd.Component(MyWaveform, X_WAVE_PV)
    signal = ophyd.Component(MyWaveform, Y_WAVE_PV)
    
    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None
        self.t0 = time.time()
        self.waves = (self.time, self.axis, self.signal)

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self.complete_status = ophyd.DeviceStatus(self.busy)
        
        def cb(*args, **kwargs):
            if self.busy.value in (BusyStatus.done):
                self.complete_status._finished(success=True)
        
        self.t0 = time.time()
        self.busy.put(BusyStatus.busy)
        self.busy.subscribe(cb)

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete(): " + str(self.complete_status))
        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        schema = {}
        for item in self.waves:
            structure = dict(
                source = item.wave.pvname,
                dtype = "number",
                shape = (1,)
            )
            schema[item.name] = structure
        return {self.name: schema}

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect(): " + str(self.complete_status))
        self.complete_status = None
        for i in range(int(self.time.number_read.value)):
            data = {}
            timestamps = {}
            t = time.time()
            for item in self.waves:
                data[item.name] = item.wave.value[i]
                timestamps[item.name] = t
            data[self.time.name] -= self.t0  # demo: offset time (removes large offset)
            d = dict(
                time=time.time(),
                data=data,
                timestamps=timestamps
            )
            yield d

In [5]:
bflyer = BusyFlyerDevice(name="bflyer")

In [6]:
status = bflyer.complete()
status

In [7]:
list(bflyer.collect())

[{'data': {'bflyer_axis': -1.23,
   'bflyer_signal': 0.0056610971236743723,
   'bflyer_time': -102.14657592773438},
  'time': 1524924664.4834409,
  'timestamps': {'bflyer_axis': 1524924664.4755518,
   'bflyer_signal': 1524924664.4755518,
   'bflyer_time': 1524924664.4755518}},
 {'data': {'bflyer_axis': 0.87,
   'bflyer_signal': 0.87074082551308463,
   'bflyer_time': -99.743170022964478},
  'time': 1524924664.4839718,
  'timestamps': {'bflyer_axis': 1524924664.483443,
   'bflyer_signal': 1524924664.483443,
   'bflyer_time': 1524924664.483443}},
 {'data': {'bflyer_axis': 2.9700000000000002,
   'bflyer_signal': 0.017715724422064545,
   'bflyer_time': -97.339380979537964},
  'time': 1524924664.484477,
  'timestamps': {'bflyer_axis': 1524924664.4839733,
   'bflyer_signal': 1524924664.4839733,
   'bflyer_time': 1524924664.4839733}},
 {'data': {'bflyer_axis': 5.0700000000000003,
   'bflyer_signal': 0.35019455252918286,
   'bflyer_time': -94.935523986816406},
  'time': 1524924664.4863439,
  't

In [8]:
import bluesky.utils
RE.msg_hook = bluesky.utils.ts_msg_hook
logging.basicConfig(
    # level=logging.INFO, 
    level=logging.DEBUG, 
    format='%(asctime)s.%(msecs)03d %(levelname)s %(message)s',
    datefmt='%M:%S',
    # datefmt='%Y-%m-%d %H:%M:%S',
    ) 

In [9]:
RE(bluesky.plans.fly([bflyer]))

11:04.550 DEBUG Inserted RunStart with uid 35ace6f9-521b-4247-a890-fb0c3306e770
11:04.555 INFO kickoff()
11:04.571 INFO complete(): DeviceStatus(device=bflyer_busy, done=False, success=False)


09:11:04.532076 open_run          -> None            args: (), kwargs: {}
09:11:04.552720 kickoff           -> bflyer          args: (), kwargs: {'group': None}
09:11:04.570638 wait              -> None            args: (), kwargs: {'group': None}
09:11:04.571154 complete          -> bflyer          args: (), kwargs: {'group': None}
09:11:04.573744 wait              -> None            args: (), kwargs: {'group': None}


11:14.326 INFO describe_collect()
11:14.350 DEBUG Inserted EventDescriptor with uid 668b0df2-d6af-4ae1-9aca-4ce71df0cf7e referencing RunStart with uid 35ace6f9-521b-4247-a890-fb0c3306e770
11:14.353 INFO collect(): DeviceStatus(device=bflyer_busy, done=True, success=True)
11:14.376 DEBUG Inserted RunStop with uid b4b5df0a-801c-45ba-95b5-1dbf1974a73b referencing RunStart  with uid 35ace6f9-521b-4247-a890-fb0c3306e770


09:11:14.325695 collect           -> bflyer          args: (), kwargs: {'stream': False}
09:11:14.373879 close_run         -> None            args: (), kwargs: {'exit_status': None, 'reason': None}


('35ace6f9-521b-4247-a890-fb0c3306e770',)

In [10]:
# RE.abort()

In [11]:
h = db[-1]
h.table("bflyer")

11:14.411 INFO Interpreting key = -1 as an integer


,time,bflyer_time,bflyer_axis,bflyer_signal
seq_num,,,,
1,2018-04-28 10:11:14.355848,0.139265,-1.23,0.946609
2,2018-04-28 10:11:14.358687,2.544316,0.87,0.946609
3,2018-04-28 10:11:14.361532,4.954960,2.97,0.845502
4,2018-04-28 10:11:14.362693,7.361858,5.07,0.327810
5,2018-04-28 10:11:14.363223,9.766186,7.17,0.716808
